# Generación de Datos
por: Mateo Yáñez Tanaka

El ojetivo de este notebook es generar la base de datos que se usará para responder los queries de la investigación

## Procesamiento de los datos

In [12]:
# importando librerías
import numpy as np #importamos numpy para manejo de arrays
import pandas as pd #importamos pandas para el manejo de dataframes

#leyendo la base de datos
tvivienda = pd.read_csv('../Data/tvivienda.csv')
ttransporte = pd.read_csv('../Data/ttransporte.csv')
tviaje = pd.read_csv('../Data/tviaje.csv')
tsdem = pd.read_csv('../Data/tsdem.csv')
thogar = pd.read_csv('../Data/thogar.csv')

Las queries que nos tocaron fueron las siguientes:
- ¿Cuál es la probabilidad de que una mujer joven de estrato social alto use la bicicleta?
- ¿Cuál es la probabilidad de que una persona que vive en una localidad de menos de 2,500 habitantes sea de estrato alto?
- ¿Cuál es la probabilidad de que alguien de estrato sociodemográfico bajo tenga ya sea una moto, un auto o una bicicleta?
- Se busca averiguar si las mujeres de edad media y alta, de estrato social bajo tienden a caminar más entre semana en comparación a los hombres con las mismas características.

Tomando en cuenta estas queries, las variables que requeriremos de la bsase de datos serán las siguientes: 
- ```estrato```: Se encuentra en la mayoría de los archivos de datos pero para este caso lo tomaremos de ```tvivienda``` y se refiere a el estrato social de la persona. Sus categorías son las siguientes:
    1. Bajo
    2. Medio Bajo
    3. Medio Alto
    4. Alto 
- ```sexo```: Se encuentra en el archivo ```ttransporte``` y se refiere al sexo de la persona. Sus categorías son las siguientes:
    1. Hombre
    2. Mujer
- ```p5_14```: Se encuentra en el archivo ```ttransporte``` y se refiere el medio de transporte que utilizó la persona en su viaje. Sus categorías son las siguientes:
    1. Automóvil
    1. Colectivo/Micro
    1. Taxi (App Internet)
    1. Taxi (sitio, calle u otro)
    1. Metro
    1. Autobús RTP o M1
    1. Bicicleta
    1. Autobús
    1. Moto
    1. Trolebús
    1. Metroús o Mexibús
    1. Tren ligero
    1. Tren suburbano
    1. Caminar en la calle
    1. Mexicable
    1. Bicitaxi
    1. Mototaxi
    1. Transporte escolar
    1. Transporte de personal
    1. Otro
- ```edad``` se encuentra en ```ttransporte``` y se refiere a la edad de la persona, cubre el rango de 6 a 99 años de edad.
- ```p2_1_1``` se encuentra en ```thogar``` y se refiere a la cantidad de autos que poseé la persona para su transpoorte cotidiano. Toma valores entre 0-8, con 0 siendo cero carros y 8 siendo ocho o más.  
- ```p2_1_2``` se encuentra en ```thogar``` y se refiere a la cantidad de motos que poseé la persona para su transporte cotidiano. Toma valores entre 0-8, con 0 siendo cero motos y 8 siendo oocho o más. 
- ```p2_1_3``` se encuentra en ```thogar``` y se refiere a la cantidad de bicicletas que poseé la persona para su transporte cootidiano. Toma valores entre 0-8, con 0 siendo cero bicicletas y 8 siendo ocho o más.
- ```tloc``` se encuentra en ```tvivienda``` y se refiere a la categooría de la localidad basado en su cantidad de población. Las categorías son las siguientes:
    1. localidades mayores a 100,000 habitantes
    2. localidades de 15,000 a 99,000 habitantes
    3. localidades de 2,500 a 14,999 habitantes
    4. loclaidades de menos de 2,500 habitantes
- ```p5_3``` se encuentra en ```tviaje``` y refiere al tipo de día en el que realizó el viaje. Sus categorías son las siguientes:
    1. Entre semana
    2. Sábado

In [13]:
# Unimos las columnas que nos interesan en una sola base de datoos
data = pd.merge(pd.merge(pd.merge(pd.merge(tvivienda[['tloc','estrato','id_viv']], thogar[['id_viv','p2_1_1','p2_1_2','p2_1_3','id_hog']], on='id_viv', how = 'inner'), tsdem[['id_hog','id_soc']], on = 'id_hog',how = 'inner'),tviaje[['p5_3','id_soc','id_via']],on='id_soc',how = 'inner'), ttransporte[['id_via','edad','p5_14','sexo']],on = 'id_via', how = 'inner')
data.drop(columns = ['id_hog','id_soc','id_via'],inplace=True)

Con el DataFrame con nuestras variables ya creado, nos aseguramos de que los datos se encuentren limpios, sin valores vacíos y con todos sus datos tipificados propiamente.

In [14]:
#Verificamos que no existan valoores nulos en las columnas
for col in data.columns:
    n = data[col].isna().mean()
    if n > 0:
        print(f'La columna {col} tiene en promedio {n} valores nulos')

#Verificamos que los datos estén propiamente tipificados
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 890748 entries, 0 to 890747
Data columns (total 10 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   tloc     890748 non-null  int64
 1   estrato  890748 non-null  int64
 2   id_viv   890748 non-null  int64
 3   p2_1_1   890748 non-null  int64
 4   p2_1_2   890748 non-null  int64
 5   p2_1_3   890748 non-null  int64
 6   p5_3     890748 non-null  int64
 7   edad     890748 non-null  int64
 8   p5_14    890748 non-null  int64
 9   sexo     890748 non-null  int64
dtypes: int64(10)
memory usage: 68.0 MB


La tipifícación de datos es correcta, sin embargo, para las queries que necesitamos realizar no nos funciona la forma en la que la edad está tipificada. Necesitamos que sean únicamente categorías del tipo "adultez mayor", "adultez media" "adultez joven", "adolescencia" y "niñez" para lo cuál usaremos las siguientes categorías
1. niñez $6 \le x \le 12$
2. adolescencia $13 \le x \le 17$
3. adultez joven $18 \le x \le 39$
4. adultez media $40 \le x \le 59$
5. adultez mayor $60 \le x \le 99$

In [20]:
bins = [5,12,17,39,59,99]
labels = [ k for k in range(1,len(bins))]

data['edad'] = pd.cut(data['edad'],bins = bins, labels=labels, right = True)

Dado que no se presentaron valores nulos en ninguna de las columnas, y habiendo corregido la tipificación de la edad, guardaremos los datos en un archivo llamado ```datos.csv```

In [24]:
#convertimos nuestro pandas dataframe a un archivo CSV
data.to_csv('../Data/datos.csv') 